In [26]:

import sys
sys.path.append('/home/nick/work/MLIR/build/tools/mlir/python_packages/mlir_core')
sys.path.append('/home/nick/work/mlir_nelli')
sys.path.append('/home/nick/work/mlir_nelli/.venv/lib/python3.11/site-packages')


import numpy as np

import mlir.extras.types as T
from mlir.extras.ast.canonicalize import canonicalize
from mlir.extras.context import ContextManager
from mlir.extras.dialects.ext.arith import constant
from mlir.extras.dialects.ext.memref import S
from mlir.extras.dialects.ext.func import toMLIRFunc
from mlir.extras.dialects.ext.scf import canonicalizer as scf_canonicalizer, range_ as range
from mlir.extras.runtime.passes import Pipeline, run_pipeline
from mlir.extras.runtime.refbackend import LLVMJITBackend
from mlir.ir import StridedLayoutAttr
from mlir.extras.dialects.ext import linalg
from mlir.dialects.transform.extras import named_sequence
from mlir.dialects import pdl
from mlir.dialects.transform.structured import structured_match
from mlir.dialects.transform.loop import loop_unroll
from mlir.ir import UnitAttr
from mlir.dialects.transform import get_parent_op, print_, any_op_t
from mlir.dialects.builtin import module
from mlir.extras.dialects.ext.transform import (
    match,
    tile,
    tile_to_scf_forall,
)
# import IREE binding
from iree import runtime as ireert
from iree.compiler import compile_str

import re

# create MLIR context
CM = ContextManager()
Ctx = CM.__enter__()
backend = LLVMJITBackend()

K = 1024

@toMLIRFunc(emit=True)
@canonicalize(using=scf_canonicalizer)
def matmul(
    arg0: T.tensor(K, K, T.f32()),
    arg1: T.tensor(K, K, T.f32()),
    out : T.tensor(K, K, T.f32()),
):
    return linalg.matmul(arg0, arg1, out)


module = str(run_pipeline(Ctx.module, Pipeline().cse()))

# dirty work to rename module
pattern = re.compile(r'(module\s*\{)')
module= re.sub(pattern, r'module @arithmetic {', module)



compiled_flatbuffer = compile_str(module, target_backends=["vmvx"])
config = ireert.Config("local-task")
ctx = ireert.SystemContext(config=config)
vm_module = ireert.VmModule.from_flatbuffer(ctx.instance, compiled_flatbuffer)
ctx.add_vm_module(vm_module)

# Invoke the function and print the result.
print("INVOKE matmul")
arg0 = np.ones((K, K), dtype=np.float32)
arg1 = np.ones((K, K), dtype=np.float32)
init = np.zeros((K, K), dtype=np.float32)
f = ctx.modules.arithmetic["matmul"]
result = f(arg0, arg1, init).to_host()
print("Results:", result)
assert np.array_equal(np.dot(arg0, arg1), result)
CM.__exit__(None, None, None)


<class 'mlir._mlir_libs._mlir.ir.Module'>
module {
  func.func @matmul(%arg0: tensor<3x3xf32>, %arg1: tensor<3x3xf32>, %arg2: tensor<3x3xf32>) -> tensor<3x3xf32> {
    %0 = linalg.matmul {cast = #linalg.type_fn<cast_signed>} ins(%arg0, %arg1 : tensor<3x3xf32>, tensor<3x3xf32>) outs(%arg2 : tensor<3x3xf32>) -> tensor<3x3xf32>
    return %0 : tensor<3x3xf32>
  }
}

INVOKE matmul
Results: [[3. 3. 3.]
 [3. 3. 3.]
 [3. 3. 3.]]


/tmp/ipykernel_3139033/722318355.py:66: UserWarning: Making copy of unaligned VmModule buffer. It is recommended to make this deterministic by calling `copy_buffer` to always make a copy or `mmap` to efficiently load from a file. This warning can be silenced by adding `warn_if_copy=False` to `from_buffer`
  vm_module = ireert.VmModule.from_flatbuffer(ctx.instance, compiled_flatbuffer)


False

In [2]:


#@module(attrs={"transform.with_named_sequence": UnitAttr.get()})
#def mod():
#    @named_sequence("__transform_main", [any_op_t()], [])
#    def sched(target):
#        m = match(target, ["linalg.matmul"])
#       tiled_linalg_op, loops = tile(m, sizes=[8, 32])

#module = run_pipeline(Ctx.module, Pipeline().transform_interpreter().canonicalize())
#print(Ctx.module)
#invoker = backend.load(module)
#A = np.random.randint(0, 10, (K, K)).astype(np.float32)
#AA = ctypes.pointer(ctypes.pointer(get_ranked_memref_descriptor(A)))
#B = np.random.randint(0, 10, (K, K)).astype(np.float32)
#BB = ctypes.pointer(ctypes.pointer(get_ranked_memref_descriptor(B)))
#C = np.zeros((K, K)).astype(np.float32)
#CC = ctypes.pointer(ctypes.pointer(get_ranked_memref_descriptor(C)))

#invoker.memfoo_capi_wrapper(AA, BB, CC)
#assert np.array_equal(A + B, C)


module {
  func.func @matmul(%arg0: tensor<1024x1024xf32>, %arg1: tensor<1024x1024xf32>, %arg2: tensor<1024x1024xf32>) -> tensor<1024x1024xf32> attributes {llvm.emit_c_interface} {
    %0 = linalg.matmul {cast = #linalg.type_fn<cast_signed>} ins(%arg0, %arg1 : tensor<1024x1024xf32>, tensor<1024x1024xf32>) outs(%arg2 : tensor<1024x1024xf32>) -> tensor<1024x1024xf32>
    return %0 : tensor<1024x1024xf32>
  }
}



MlirCompilerError: Lowering IR failed with the following diagnostics:

********************************************************************************
Failure while executing pass pipeline:
error: "-":3:10: expected add/mul op in the body
note: "-":3:10: see current operation: 
"linalg.matmul"(%8, %17, %52) <{cast = #linalg.type_fn<cast_signed>, operandSegmentSizes = array<i32: 2, 1>}> ({
^bb0(%arg21: f32, %arg22: f32, %arg23: f32):
%64 = "llvm.fmul"(%arg21, %arg22) <{fastmathFlags = #llvm.fastmath<none>}> : (f32, f32) -> f32
%65 = "llvm.fadd"(%arg23, %64) <{fastmathFlags = #llvm.fastmath<none>}> : (f32, f32) -> f32
"linalg.yield"(%65) : (f32) -> ()
}) {linalg.memoized_indexing_maps = [affine_map<(d0, d1, d2) -> (d0, d2)>, affine_map<(d0, d1, d2) -> (d2, d1)>, affine_map<(d0, d1, d2) -> (d0, d1)>]} : (memref<1024x1024xf32>, memref<1024x1024xf32>, memref<1024x1024xf32>) -> ()
********************************************************************************

For developers, the error can